In [ ]:
from __future__ import print_function, division, absolute_import 
import colamatch as clm
import numpy as np
import cv2
from matplotlib import pyplot as plt
import time
%matplotlib notebook
import logging
import sys
logging.basicConfig(
        stream=sys.stdout,
        level=logging.INFO,
        format='%(asctime)s %(name)s [%(levelname)s]:%(message)s',
        datefmt='%Y-%m-%d %H:%M:%S')

## Define filenames and read images and pointsets

In [ ]:
# choose testset:
testset = 2 # available testsets: 1, 2

testset2img_ids = {1:[1,2], 2:[11,12]}
s1, s2 = testset2img_ids[testset]
img_fname1 = "../data/section_%s.tif"%(str(s1).zfill(4))
img_fname2 = "../data/section_%s.tif"%(str(s2).zfill(4)) 
points_fname1 = "../data/section_%s.landmarks.txt"%(str(s1).zfill(4))
points_fname2 = "../data/section_%s.landmarks.txt"%(str(s2).zfill(4))

In [ ]:
# read pointsets
num_points = -1  # restrict number of points given into matching (-1 for all points)
pointlist1 = np.loadtxt(points_fname1).astype('int')[:num_points]
pointlist2 = np.loadtxt(points_fname2).astype('int')[:num_points]
print(len(pointlist1), len(pointlist2))

## Constellation matching

### Parameters for constellation matching:
* **num_samples**: Number of hashes added to KDTree for pointlists in 1st and 2nd image (depends on #num_points -> more samples needed for increasing number of points used for matching)
* **coordinate_weight**: Weight for absolute point coordinates in hash code.
* **ransac**: Hessian threshold for homography ransac on matched normalized point coordinates (0.01 works good)
* **radius**: Radius for finding similar hashes in KDTree.

In [ ]:
num_samples = 10000
sampler1 = clm.RandomSampler(len(pointlist1), 4, num_samples)
sampler2 = clm.RandomSampler(len(pointlist2), 4, num_samples)
start = time.time()
matches = clm.match(pointlist1, pointlist2, sampler1, sampler2, 
                    radius=0.1, coordinate_weight=2, ransac=0.01)
print("Runtime for num_samples=%s: %f" % (num_samples,time.time()-start))

## Plot results

In [ ]:
# read images
img1 = cv2.imread(img_fname1, 0)
img2 = cv2.imread(img_fname2, 0)
# plot points and matches into images
cmap = plt.cm.get_cmap("hsv", len(matches))
fig,axs = plt.subplots(2,1)
axs[0].imshow(img1, cmap='gray')
axs[1].imshow(img2, cmap='gray')
axs[0].plot(pointlist1[:,0], pointlist1[:,1],'w+',ms=15)
axs[1].plot(pointlist2[:,0], pointlist2[:,1],'w+',ms=15)
axs[0].axis('off')
axs[1].axis('off')
for i,candidate in enumerate(matches): 
    axs[0].plot(candidate[0,0], candidate[0,1], c=cmap(i), marker='o')
    axs[1].plot(candidate[1,0], candidate[1,1], c=cmap(i), marker='o')
fig.show()